# mesoSPIM PSF-analysis

Currently, this analysis notebook needs a stack with beads converted to `.tif`-files. pixel-size and zoom have to be set manually.

### Setup environment

In [1]:
import numpy as np
import pandas as pd
from skimage.io import imread, find_available_plugins

In [2]:
from psf import compute, plotPSF

### Setup plotting

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('paper', font_scale=2.0)
sns.set_style('ticks')

In [4]:
from ipywidgets import interact
from ipywidgets import IntSlider

### Define parameters

`windowUm = [12, 10, 10]` defines a "window" volume in microns in which only a single bead can exist (for the bead to be included in the analysis, the volume has to contain a single bright maximum).

In [ ]:
# Full FOV
FOVpxLat = 2048.0 # 2048
UmPerPxLat = 1.6
pxPerUmLat = 1/UmPerPxLat
FOVumLat = FOVpxLat / pxPerUmLat
pxPerUmAx = 1.0 # 1.0
windowUm = [12, 10, 10]
options = {'FOVumLat':FOVumLat, 'FOVpxLat':FOVpxLat, 'pxPerUmLat':FOVpxLat/FOVumLat, 'pxPerUmAx':pxPerUmAx, 'windowUm':windowUm}
options['thresh'] = .05

In [5]:
# 1/4 FOV
FOVpxLat = 512.0 # 2048
UmPerPxLat = 1.6
pxPerUmLat = 1/UmPerPxLat
FOVumLat = FOVpxLat / pxPerUmLat
pxPerUmAx = 1.0 # 1.0
windowUm = [20, 5, 5]
options = {'FOVumLat':FOVumLat, 'FOVpxLat':FOVpxLat, 'pxPerUmLat':FOVpxLat/FOVumLat, 'pxPerUmAx':pxPerUmAx, 'windowUm':windowUm}
options['thresh'] = .05

In [ ]:
options

### Load data

In [ ]:
im = imread('./data/images.tif', plugin='tifffile') # old testdata

In [ ]:
im = imread('./data/test2-withETL.tif', plugin='tifffile') # 512 px testdata

In [ ]:
im = imread('./data/test2-withETL.tif') # 512 px testdata

In [ ]:
im = imread('./data/5ms_2nd_488_nm_508_520-35_4x_Right_000000-3-200slices.tif', plugin='tifffile')  # 2048 px testdata

In [ ]:
im = imread('./data/5ms_2nd_noETL_488_nm_508_520-35_4x_Right_000000-2-200slices.tif', plugin='tifffile')  # 2048 px testdata

### Compute

In [ ]:
data, beads, maxima, centers, smoothed = compute(im, options)

In [ ]:
centers = pd.DataFrame(centers, columns=['Z','Y','X'])

In [ ]:
PSF = pd.concat([x[0] for x in data])
PSF['Max'] = maxima
PSF = PSF.reset_index().drop(['index'],axis=1)
latProfile = [x[1] for x in data]
axProfile = [x[2] for x in data]
PSF = PSF.join(centers)

### Inspect results

In [ ]:
PSF.head()

In [ ]:
print('# Beads: ', len(PSF))
print('Mean Lateral FWHM: ', round(PSF['FWHMlat'].mean(),3), ' +/- ', round(PSF['FWHMlat'].sem(),3), ' μm')
#print('STD lateral FWHM: ', round(PSF['FWHMlat'].std(),3))
print('Mean axial FWHM: ', round(PSF['FWHMax'].mean(),3), ' +/- ', round(PSF['FWHMax'].sem(),3), ' μm')
#print('STD axial FWHM: ', round(PSF['FWHMax'].std(),3))

### Plot axial FWHM vs. FOV

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15,10));

subfigure0 = fig.add_subplot(211)
subfigure0.plot(np.multiply(PSF['Y'].tolist(),UmPerPxLat),PSF['FWHMax'].tolist(),'.b',ms=10)
subfigure0.set_xlim([0,options['FOVumLat']])
subfigure0.set_ylim([0,25])
subfigure0.set_xlabel('Y Distance (μm)')
subfigure0.set_ylabel('Axial FWHM (μm)')

subfigure1 = fig.add_subplot(212)
subfigure1.plot(np.multiply(PSF['X'].tolist(),UmPerPxLat),PSF['FWHMax'].tolist(),'.r',ms=10)
subfigure1.set_xlim([0,options['FOVumLat']])
subfigure1.set_ylim([0,25])
subfigure1.set_xlabel('X Distance (μm)')
subfigure1.set_ylabel('Axial FWHM (μm)')

plt.subplots_adjust(hspace = 0.3)

### Save data and plots

In [ ]:
PSF.to_csv('results/WithETL.csv')

In [ ]:
fig.savefig('results/ETL-Comparision.eps', facecolor=fig.get_facecolor(), edgecolor='none',dpi=200)
fig.savefig('results/ETL-Comparision.svg', facecolor=fig.get_facecolor(), edgecolor='none',dpi=200)

## Sanity checks

### Plot max projection and detected beads

In [ ]:
plt.figure(figsize=(10,10));
plt.imshow(smoothed);
plt.plot(PSF['X'].tolist(), PSF['Y'].tolist(), 'g.', ms=5);
plt.xlim([0, smoothed.shape[0]])
plt.ylim([smoothed.shape[1], 0])
plt.axis('off'); 

### Axial and lateral FWHM Histogram

In [ ]:
fig = plt.figure(figsize=(10,5))
subfigure0 = fig.add_subplot(121)
subfigure0.hist(PSF['FWHMax'].tolist())
subfigure0.set_xlabel('Axial FWHM (μm)')
subfigure0.set_ylabel('# Beads')
subfigure0 = fig.add_subplot(122)
subfigure0.hist(PSF['FWHMlat'].tolist())
subfigure0.set_xlabel('Lateral FWHM (μm)')
subfigure0.set_ylabel('# Beads')

plt.subplots_adjust(wspace = 0.5)

### Lateral vs. axial FWHM correlation

In [ ]:
fig = plt.figure(figsize=(10,10))
subfigure0 = fig.add_subplot(111)
subfigure0.scatter(PSF['FWHMlat'].tolist(),PSF['FWHMax'].tolist())
subfigure0.set_xlabel('Lateral FWHM (μm)')
subfigure0.set_ylabel('Axial FWHM (μm)')

### Overview Plots

In [ ]:
fig = plt.figure(figsize=(15,7.5));

subfigure0 = fig.add_subplot(121)
subfigure0.imshow(smoothed);
overlay0 = subfigure0.scatter(PSF['X'].tolist(), PSF['Y'].tolist(), c=PSF['FWHMax'].tolist(), cmap='jet');
subfigure0.axis('off');
subfigure0.set_title('Axial FWHM')
cbar0 = plt.colorbar(overlay0,fraction=0.044, pad=0.04)
cbar0.set_label('Axial FWHM (μm)')


subfigure1 = fig.add_subplot(122)
subfigure1.imshow(smoothed);
overlay1 = subfigure1.scatter(PSF['X'].tolist(), PSF['Y'].tolist(), c=PSF['FWHMlat'].tolist(), cmap='jet');
subfigure1.axis('off');
subfigure1.set_title('Lateral FWHM')
cbar1 = plt.colorbar(overlay1,fraction=0.044, pad=0.04)
cbar1.set_label('Lateral FWHM (μm)')


### Plot XYZ projections and fit

In [ ]:
fig = plt.figure()

def update(w = 50):
    beadInd = w
    average = beads[beadInd]
    
    fig, _axs = plt.subplots(nrows=1, ncols=3)
    axs = _axs.flatten()

    XYview = axs[0].imshow(average.mean(axis=0));
    XZview = axs[1].imshow(average.mean(axis=1), aspect = pxPerUmLat/pxPerUmAx);
    YZview = axs[2].imshow(average.mean(axis=2), aspect = pxPerUmLat/pxPerUmAx);
    plt.show()
    plotPSF(latProfile[beadInd][0],latProfile[beadInd][1],latProfile[beadInd][2],latProfile[beadInd][3],pxPerUmLat,PSF.Max.iloc[beadInd])
    plt.show()
    plotPSF(axProfile[beadInd][0],axProfile[beadInd][1],axProfile[beadInd][2],axProfile[beadInd][3],pxPerUmAx,PSF.Max.iloc[beadInd])
    plt.show()

interact(update, w=IntSlider(min=0,max=len(PSF)-1,step=1,value=0,continuous_update=False));